In [2]:
print("ok")

ok


In [3]:
%pwd

'c:\\medical\\medical-chatbot-AI\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'c:\\medical\\medical-chatbot-AI'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [8]:
extracted_data = load_pdf_file(data='C:\medical\Data')

In [9]:
#extracted_data

In [10]:
# Split the Data into Text Chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter 

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)

print("Length of Text Chunks:", len(text_chunks))

Length of Text Chunks: 5859


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings


In [13]:
# 1. Define the function (you already did this)
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [14]:
pip install --upgrade sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [15]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')


c:\Users\kisho\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
embeddings = download_hugging_face_embeddings()


C:\Users\kisho\AppData\Local\Temp\ipykernel_18296\1038990354.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [18]:
#query_result

In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
PINECONE_API_KEY=os.environ.get('pcsk_4dRr3v_PyJzmcQsgBWkSb9AL9c3rMb1QDVTBy128Mbo1uQqezdCXKER2WtwsGy3iSHNNni')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [21]:
!pip install google protobuf
!pip install sentence-transformers
!pip install sentence-transformers
!pip install -U langchain-community
!pip install --upgrade protobuf==6.31.0
!pip install pinecone
!pip install langchain_pinecone


In [22]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec




# Initialize Pinecone client with API key
pc = Pinecone(api_key="pcsk_4dRr3v_PyJzmcQsgBWkSb9AL9c3rMb1QDVTBy128Mbo1uQqezdCXKER2WtwsGy3iSHNNni")

# Define index name and configuration
index_name = "medibot-v2"

# Create the index
pc.create_index(
    name=index_name,
    dimension=384,  # must match your embedding size
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

c:\Users\kisho\anaconda3\envs\medibot\lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.0 at db_data_2025-01.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'ff70cd122343c674193346162937df45', 'date': 'Thu, 19 Jun 2025 10:53:10 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [23]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_4dRr3v_PyJzmcQsgBWkSb9AL9c3rMb1QDVTBy128Mbo1uQqezdCXKER2WtwsGy3iSHNNni"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [24]:
from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [25]:
from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    
    index_name=index_name,
    embedding=embeddings)

In [26]:
docsearch

In [27]:
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # fetch top 3 similar documents
)

# Invoke the retriever with a query
retrieved_docs = retriever.invoke("What is Acne?")


In [28]:
retrieved_docs

[Document(id='42fea941-b59f-4e29-878c-d5e2e0c69134', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\medical\\Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='f4a8904e-0db4-4ec9-803d-142d711ef441', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\medical\\Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='803d091c-4ab2-4837-8679-d8bf45251fab', metadata={'creationdate': '2004-12-18T17:00

In [29]:
#brief record from book

In [30]:
for i, doc in enumerate(retrieved_docs):
    print(f"\nDocument {i+1}:\n")
    print(doc.page_content)


Document 1:

GALE ENCYCLOPEDIA OF MEDICINE 226
Acne
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26

Document 2:

GALE ENCYCLOPEDIA OF MEDICINE 226
Acne
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26

Document 3:

GALE ENCYCLOPEDIA OF MEDICINE 226
Acne
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26


In [31]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.4,
    max_tokens=300,
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",  # or any supported Together model
    openai_api_key="f99e4e137a324ca884a38408a1991c00c7ec67fbe53b5fce65ddb317e71c5a8b",
    openai_api_base="https://api.together.xyz/v1"
)


C:\Users\kisho\AppData\Local\Temp\ipykernel_18296\18895536.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [32]:
from langchain.chains import create_retrieval_chain
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
  # or any other chat model you are using

# Define the system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\nContext:\n{context}"
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [33]:
!pip install -U langchain-openai



  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.3.23
    Uninstalling langchain-openai-0.3.23:
      Successfully uninstalled langchain-openai-0.3.23


In [34]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain= create_retrieval_chain(retriever, question_answer_chain)


In [35]:
response = llm.invoke("What is acne?")
print(response.content)


 Acne is a common skin condition that occurs when the hair follicles become plugged with oil and dead skin cells, leading to whiteheads, blackheads, or pimples. It primarily affects the face, forehead, chest, upper back, and shoulders. Acne can occur at any age, but it is most common among teenagers and young adults.

Acne develops when the skin's oil glands produce an excess amount of sebum, a natural oil that keeps the skin moisturized. When there is too much sebum, the dead skin cells that are normally shed from the skin can stick together and clog the pores. Bacteria that are normally present on the skin can also get trapped in the pores and cause inflammation, leading to the formation of pimples.

There are several factors that can contribute to the development of acne, including hormonal changes, genetics, stress, and the use of certain medications. Treatment for acne depends on the severity of the condition and can include over-the-counter or prescription topical medications, an

In [37]:
response = llm.invoke("who is virat kholi?")
print(response.content)


 Virat Kohli is a professional cricket player from India, who is widely regarded as one of the best batsmen in the world. He was born on November 5, 1988, in Delhi, India.

Kohli made his debut for the Indian cricket team in 2008 and has since established himself as a prolific run-scorer in all formats of the game. He is known for his aggressive batting style, excellent footwork, and exceptional ability to score runs under pressure.

Kohli has led the Indian cricket team as its captain in all formats of the game and is known for his tactical acumen and ability to motivate his team. He has led India to numerous victories, including the 2011 Cricket World Cup and the 2013 Champions Trophy.

Off the field, Kohli is known for his philanthropy and his support for various social causes. He is married to the Bollywood actor Anushka Sharma, and the couple is often in the limelight for their public appearances and endorsements.

Kohli has won numerous awards for his cricketing prowess, includin